# Backtest benchmark

In this notebook we will perform GARCH backtesting with Python to test speed and accuracy.

## Single timed backtest

In [1]:
# Load data in Python

import numpy as np

price = np.loadtxt('index.csv',delimiter=',',skiprows=1)

y = np.diff(np.log(price), n=1, axis=0) 

In [2]:
# Set backtest up in Python

T = len(y)                   # number of obs for y
p = 0.01                     # probability
value = 1                    # portfolio value
VaR = np.full(T, np.nan) # matrix for forecasts

In [34]:
# Running backtest in Python

from scipy import stats
from arch import arch_model

def backtest(WE):
    VaR = np.full(T, np.nan)
    for t in range(WE, T): 
        t1 = t - WE           # start of data window
        t2 = t - 1            # end of data window
        window = y[t1:t2+1]   # data for estimation


        am = arch_model(window, mean = 'Zero',vol = 'Garch',
                        p = 1, o = 0, q = 1, dist = 'Normal', rescale = False)
        res = am.fit(update_freq=0, disp = 'off', show_warning=False)
        par = [res.params[0], res.params[1], res.params[2]]
        s4 = par[0] + par[1] * window[WE - 1]**2 + par[
            2] * res.conditional_volatility[-1]**2
        VaR[t] = -np.sqrt(s4) * stats.norm.ppf(p) * value 
    return(VaR)

In [35]:
# Benchmark

import time

start_time = time.time()
VaR_results = backtest(WE = 2000)
elapsed = time.time() - start_time
print("Time elapsed for one iteration using WE = 2000:", elapsed, "seconds")

Time elapsed for one iteration using WE = 2000: 49.98308706283569 seconds


In [36]:
# Export results
np.savetxt("python_backtest_results.csv", VaR_results, delimiter=",")

## Get average of N backtests using WE = 2000

In [42]:
# Average simulation time
N = 10
WE_2000 = np.full(N, np.nan)
for i in range(N):
    start_time = time.time()
    backtest(WE = 2000)
    WE_2000[i] = time.time() - start_time

In [46]:
print("Average time elapsed for", N, "iterations using WE = 2000:", np.mean(WE_2000), "seconds")

Average time elapsed for 10 iterations using WE = 2000: 43.00818808078766 seconds
